<a href="https://colab.research.google.com/github/sanaaria/Big-data/blob/main/big_data_project_by_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=7d8a6957bc4386726c29f247394b69b33d8bc6303470887d72a244917f258743
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
import csv
from google.colab import files
 
 
uploaded = files.upload()

Saving reddit_data.csv to reddit_data.csv


In [3]:
# import pandas as pd
# import io
 
# #data = pd.read_csv(io.BytesIO(uploaded['reddit_data.csv']))

In [4]:
data="/content/reddit_data.csv"

In [5]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
df = spark.read.csv(data, inferSchema=True, header=False)

In [7]:

columns = ["parent_id","text","topic","length","size_range"]
data = df.toDF(*columns)


In [8]:
data.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           parent_id|                text|               topic|              length|          size_range|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|           parent_id|                text|               topic|              length|          size_range|
|          t1_crojgfu|Thanks! Not sure ...|        pcmasterrace|                 103|          101 to 200|
|          t1_cquq97y|I think its unlik...|                news|                 163|          101 to 200|
|          t1_cr92xnl|Hoult is another ...|                null|                null|                null|
|     But most of all| Fassbender and M...|                 yes|     I agree this. "|              movies|
|          t1_crk6sin|Can have my opinion.|                null|                null|                null|
|They're noisy and...| problems with 

In [9]:
from pyspark.sql.functions import col
data.select('topic').groupBy('topic').count().orderBy(col('count').desc()).show(10)

+--------------------+-----+
|               topic|count|
+--------------------+-----+
|                null|32578|
|                 nfl| 8271|
|        pcmasterrace| 7008|
|                news| 6490|
|              movies| 4615|
|       relationships| 2953|
| in fact they may...|   89|
|             rebates|   42|
|                  58|   23|
|                  63|   22|
+--------------------+-----+
only showing top 10 rows



باید داده ها به صورت عدد باشند پس در اینجا ما از الگوریتم زیر برایتبدیل سترینگ ها به عدد استفاده میکنم

In [56]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
# import org.apache.spark.sql.types.IntegerType

# Convert String to Integer Type
col("text").cast(IntegerType())

Column<'CAST(text AS INT)'>

In [57]:
col("parent_id").cast(IntegerType())

Column<'CAST(parent_id AS INT)'>

In [58]:
numeric_only = data.drop("protocol_type", "service", "flag").cache()

In [60]:
numeric_only.columns


['parent_id', 'text', 'topic', 'length', 'size_range']

In [74]:
col("parent_id").type

Column<'parent_id[type]'>

In [78]:
type(col("parent_id"))

pyspark.sql.column.Column

In [61]:
import numpy as np

In [24]:

def one_hot_encode(arr, n_labels):
    
   
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
  
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
 
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [25]:
# chars = tuple(set(text))

In [62]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans, KMeansModel 
from pyspark.ml import Pipeline

In [67]:
assembler = VectorAssembler().setInputCols(numeric_only.columns[:-4]).setOutputCol('featureVector')

In [68]:
# assembler = VectorAssembler().setInputCols(data.columns[:-5]).setOutputCol('featureVector')

In [69]:
kmeans = KMeans().setPredictionCol("cluster").setFeaturesCol("featureVector")

In [71]:
kmeans =kmeans.setK(2)
model = Pipeline(stages=[assembler, kmeans]).fit(numeric_only)
prediction = model.transform(numeric_only)

IllegalArgumentException: ignored